<a href="https://colab.research.google.com/github/YonggunJung/Project2-Review/blob/main/Electra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy
!pip install tensorflow

  Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)
  Using cached JPype1-1.4.0-cp39-cp39-win_amd64.whl (344 kB)


In [ ]:
import json
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from transformers import ElectraTokenizer
import tensorflow as tf
# 현재 프로젝트에서 GPU메모리 사용을 필요한 만큼만 쓸수 있다록 함
# 컴퓨터에 있는 GPU정도들을 가져온다.
gpus = tf.config.experimental.list_physical_devices('GPU')
# gpu가 있다면
if len(gpus) >0 :
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

Physical devices cannot be modified after being initialized


In [ ]:
df_dev = pd.read_json('./data/nikluge-sa-2022-dev.jsonl', lines=True)
df_dev

,id,sentence_form,annotation
0,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,"[[본품#편의성, [부직포 포장, 5, 11], positive]]"
1,nikluge-sa-2022-dev-00002,목욕할 때마다 넣어봤는데(샤워는 자주 해도 목욕은 그렇게 자주가 아님.. 이것도 약...,"[[본품#일반, [한약풀 냄새, 69, 75], positive]]"
2,nikluge-sa-2022-dev-00003,혹시 오래된 거 팔지 않나 고민했었는데 쑥향기 자체가 페퍼민트처럼 신선하고 포장도 ...,"[[본품#일반, [쑥향기, 22, 25], positive], [패키지/구성품#디자..."
3,nikluge-sa-2022-dev-00004,전에는 목욕할 때 입욕제 넣고 하는 거 좋아하지 않았는데 한약재로 바꿔보니 약용까지...,"[[제품 전체#품질, [한약재, 32, 35], positive]]"
4,nikluge-sa-2022-dev-00005,나 또 시계 좋아하잖아.ㅋㅋ,"[[제품 전체#일반, [시계, 4, 6], positive]]"
...,...,...,...
2789,nikluge-sa-2022-dev-02790,썸머세트라고 해서 세럼 + 선크림 이렇게 패키지로 파는 것 괜찮음,"[[패키지/구성품#일반, [썸머세트, 0, 4], positive]]"
2790,nikluge-sa-2022-dev-02791,비싸다.,"[[제품 전체#가격, [None, 0, 0], negative]]"
2791,nikluge-sa-2022-dev-02792,비싸지만 건강이 더 비싸니까.,"[[제품 전체#가격, [None, 0, 0], negative]]"
2792,nikluge-sa-2022-dev-02793,대형으로 샀더니 잘 맞음.,"[[제품 전체#일반, [대형, 0, 2], positive]]"


In [ ]:
df_dev['id'] = df_dev['id'].str.replace('nikluge-sa-2022-dev-', '')
df_dev

,id,sentence_form,annotation
0,00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,"[[본품#편의성, [부직포 포장, 5, 11], positive]]"
1,00002,목욕할 때마다 넣어봤는데(샤워는 자주 해도 목욕은 그렇게 자주가 아님.. 이것도 약...,"[[본품#일반, [한약풀 냄새, 69, 75], positive]]"
2,00003,혹시 오래된 거 팔지 않나 고민했었는데 쑥향기 자체가 페퍼민트처럼 신선하고 포장도 ...,"[[본품#일반, [쑥향기, 22, 25], positive], [패키지/구성품#디자..."
3,00004,전에는 목욕할 때 입욕제 넣고 하는 거 좋아하지 않았는데 한약재로 바꿔보니 약용까지...,"[[제품 전체#품질, [한약재, 32, 35], positive]]"
4,00005,나 또 시계 좋아하잖아.ㅋㅋ,"[[제품 전체#일반, [시계, 4, 6], positive]]"
...,...,...,...
2789,02790,썸머세트라고 해서 세럼 + 선크림 이렇게 패키지로 파는 것 괜찮음,"[[패키지/구성품#일반, [썸머세트, 0, 4], positive]]"
2790,02791,비싸다.,"[[제품 전체#가격, [None, 0, 0], negative]]"
2791,02792,비싸지만 건강이 더 비싸니까.,"[[제품 전체#가격, [None, 0, 0], negative]]"
2792,02793,대형으로 샀더니 잘 맞음.,"[[제품 전체#일반, [대형, 0, 2], positive]]"


In [ ]:
df_dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2794 entries, 0 to 2793
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             2794 non-null   object
 1   sentence_form  2794 non-null   object
 2   annotation     2794 non-null   object
dtypes: object(3)
memory usage: 65.6+ KB


In [ ]:
df_dev['label'] = df_dev['annotation'].str[0]
df_dev

,id,sentence_form,annotation,label
0,00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,"[[본품#편의성, [부직포 포장, 5, 11], positive]]","[본품#편의성, [부직포 포장, 5, 11], positive]"
1,00002,목욕할 때마다 넣어봤는데(샤워는 자주 해도 목욕은 그렇게 자주가 아님.. 이것도 약...,"[[본품#일반, [한약풀 냄새, 69, 75], positive]]","[본품#일반, [한약풀 냄새, 69, 75], positive]"
2,00003,혹시 오래된 거 팔지 않나 고민했었는데 쑥향기 자체가 페퍼민트처럼 신선하고 포장도 ...,"[[본품#일반, [쑥향기, 22, 25], positive], [패키지/구성품#디자...","[본품#일반, [쑥향기, 22, 25], positive]"
3,00004,전에는 목욕할 때 입욕제 넣고 하는 거 좋아하지 않았는데 한약재로 바꿔보니 약용까지...,"[[제품 전체#품질, [한약재, 32, 35], positive]]","[제품 전체#품질, [한약재, 32, 35], positive]"
4,00005,나 또 시계 좋아하잖아.ㅋㅋ,"[[제품 전체#일반, [시계, 4, 6], positive]]","[제품 전체#일반, [시계, 4, 6], positive]"
...,...,...,...,...
2789,02790,썸머세트라고 해서 세럼 + 선크림 이렇게 패키지로 파는 것 괜찮음,"[[패키지/구성품#일반, [썸머세트, 0, 4], positive]]","[패키지/구성품#일반, [썸머세트, 0, 4], positive]"
2790,02791,비싸다.,"[[제품 전체#가격, [None, 0, 0], negative]]","[제품 전체#가격, [None, 0, 0], negative]"
2791,02792,비싸지만 건강이 더 비싸니까.,"[[제품 전체#가격, [None, 0, 0], negative]]","[제품 전체#가격, [None, 0, 0], negative]"
2792,02793,대형으로 샀더니 잘 맞음.,"[[제품 전체#일반, [대형, 0, 2], positive]]","[제품 전체#일반, [대형, 0, 2], positive]"


In [ ]:
df_dev['label'] = df_dev['label'].str[-1]
df_dev

,id,sentence_form,annotation,label
0,00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,"[[본품#편의성, [부직포 포장, 5, 11], positive]]",positive
1,00002,목욕할 때마다 넣어봤는데(샤워는 자주 해도 목욕은 그렇게 자주가 아님.. 이것도 약...,"[[본품#일반, [한약풀 냄새, 69, 75], positive]]",positive
2,00003,혹시 오래된 거 팔지 않나 고민했었는데 쑥향기 자체가 페퍼민트처럼 신선하고 포장도 ...,"[[본품#일반, [쑥향기, 22, 25], positive], [패키지/구성품#디자...",positive
3,00004,전에는 목욕할 때 입욕제 넣고 하는 거 좋아하지 않았는데 한약재로 바꿔보니 약용까지...,"[[제품 전체#품질, [한약재, 32, 35], positive]]",positive
4,00005,나 또 시계 좋아하잖아.ㅋㅋ,"[[제품 전체#일반, [시계, 4, 6], positive]]",positive
...,...,...,...,...
2789,02790,썸머세트라고 해서 세럼 + 선크림 이렇게 패키지로 파는 것 괜찮음,"[[패키지/구성품#일반, [썸머세트, 0, 4], positive]]",positive
2790,02791,비싸다.,"[[제품 전체#가격, [None, 0, 0], negative]]",negative
2791,02792,비싸지만 건강이 더 비싸니까.,"[[제품 전체#가격, [None, 0, 0], negative]]",negative
2792,02793,대형으로 샀더니 잘 맞음.,"[[제품 전체#일반, [대형, 0, 2], positive]]",positive


In [ ]:
df_test = pd.read_json('./data/nikluge-sa-2022-test.jsonl', lines=True)
df_test

,id,sentence_form,annotation
0,nikluge-sa-2022-test-00001,하나 사려고 알아보는 중인데 맘에드는거 발견,[]
1,nikluge-sa-2022-test-00002,동양인 피부톤과 잘 어울리고 우아한 분위기를 풍긴다네?,[]
2,nikluge-sa-2022-test-00003,근데 이건 마르살라보다 더 지나친 색 같은데..,[]
3,nikluge-sa-2022-test-00004,나스 색조가 다 그렇지만서도 어데이셔스 라인은 진짜 색 기막히게 뽑는것 같다,[]
4,nikluge-sa-2022-test-00005,색상만 보면 이걸 어떻게 발라.. 싶겠지만 의외로 너무너무 괜찮다,[]
...,...,...,...
2122,nikluge-sa-2022-test-02123,간단한 충전으로 간편한 사용이 가능한거죠.,[]
2123,nikluge-sa-2022-test-02124,"눈을 가린 상태에서도 간편하게 조작이 가능하구요,",[]
2124,nikluge-sa-2022-test-02125,다양한 마사지로 관자놀이부터 눈주변까지 부드럽고 강력한 마사지가 진행됩니다.,[]
2125,nikluge-sa-2022-test-02126,"본체부터 케이블, 설명서까지 깔끔하게 정리되어 보관이 가능하니 더더 맘에 쏙 들어요.",[]


In [ ]:
df_test['id'] = df_test['id'].str.replace('nikluge-sa-2022-test-', '')
df_test

,id,sentence_form,annotation
0,00001,하나 사려고 알아보는 중인데 맘에드는거 발견,[]
1,00002,동양인 피부톤과 잘 어울리고 우아한 분위기를 풍긴다네?,[]
2,00003,근데 이건 마르살라보다 더 지나친 색 같은데..,[]
3,00004,나스 색조가 다 그렇지만서도 어데이셔스 라인은 진짜 색 기막히게 뽑는것 같다,[]
4,00005,색상만 보면 이걸 어떻게 발라.. 싶겠지만 의외로 너무너무 괜찮다,[]
...,...,...,...
2122,02123,간단한 충전으로 간편한 사용이 가능한거죠.,[]
2123,02124,"눈을 가린 상태에서도 간편하게 조작이 가능하구요,",[]
2124,02125,다양한 마사지로 관자놀이부터 눈주변까지 부드럽고 강력한 마사지가 진행됩니다.,[]
2125,02126,"본체부터 케이블, 설명서까지 깔끔하게 정리되어 보관이 가능하니 더더 맘에 쏙 들어요.",[]


In [ ]:
df_test['label'] = ''

In [ ]:
df_train = pd.read_json('./data/nikluge-sa-2022-train.jsonl', lines=True)
df_train

,id,sentence_form,annotation
0,nikluge-sa-2022-train-00001,둘쨋날은 미친듯이 밟아봤더니 기어가 헛돌면서 틱틱 소리가 나서 경악.,"[[본품#품질, [기어, 16, 18], negative]]"
1,nikluge-sa-2022-train-00002,"이거 뭐 삐꾸를 준 거 아냐 불안하고, 거금 투자한 게 왜 이래.. 싶어서 정이 확...","[[본품#품질, [기어 텐션, 67, 72], negative]]"
2,nikluge-sa-2022-train-00003,간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다.,"[[제품 전체#일반, [None, 0, 0], positive]]"
3,nikluge-sa-2022-train-00004,샥이 없는 모델이라 일반 도로에서 타면 노면의 진동 때문에 손목이 덜덜덜 떨리고 이...,"[[제품 전체#일반, [샥이 없는 모델, 0, 8], neutral]]"
4,nikluge-sa-2022-train-00005,안장도 딱딱해서 엉덩이가 아팠는데 무시하고 타고 있다.,"[[본품#일반, [안장, 0, 2], negative]]"
...,...,...,...
2996,nikluge-sa-2022-train-02997,(슬픔),"[[제품 전체#가격, [None, 0, 0], negative]]"
2997,nikluge-sa-2022-train-02998,보드랍고 괜찮다!,"[[제품 전체#품질, [None, 0, 0], positive]]"
2998,nikluge-sa-2022-train-02999,#일본 유니클로 질이 우리나라보다 좋은 것 같으면 기분 탓인가.......,"[[브랜드#일반, [유니클로, 4, 8], neutral]]"
2999,nikluge-sa-2022-train-03000,마지막으로 귀여워서 집어온 모자.,"[[제품 전체#디자인, [모자, 15, 17], positive]]"


In [ ]:
df_train['id'] = df_train['id'].str.replace('nikluge-sa-2022-train-', '')
df_train

,id,sentence_form,annotation
0,00001,둘쨋날은 미친듯이 밟아봤더니 기어가 헛돌면서 틱틱 소리가 나서 경악.,"[[본품#품질, [기어, 16, 18], negative]]"
1,00002,"이거 뭐 삐꾸를 준 거 아냐 불안하고, 거금 투자한 게 왜 이래.. 싶어서 정이 확...","[[본품#품질, [기어 텐션, 67, 72], negative]]"
2,00003,간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다.,"[[제품 전체#일반, [None, 0, 0], positive]]"
3,00004,샥이 없는 모델이라 일반 도로에서 타면 노면의 진동 때문에 손목이 덜덜덜 떨리고 이...,"[[제품 전체#일반, [샥이 없는 모델, 0, 8], neutral]]"
4,00005,안장도 딱딱해서 엉덩이가 아팠는데 무시하고 타고 있다.,"[[본품#일반, [안장, 0, 2], negative]]"
...,...,...,...
2996,02997,(슬픔),"[[제품 전체#가격, [None, 0, 0], negative]]"
2997,02998,보드랍고 괜찮다!,"[[제품 전체#품질, [None, 0, 0], positive]]"
2998,02999,#일본 유니클로 질이 우리나라보다 좋은 것 같으면 기분 탓인가.......,"[[브랜드#일반, [유니클로, 4, 8], neutral]]"
2999,03000,마지막으로 귀여워서 집어온 모자.,"[[제품 전체#디자인, [모자, 15, 17], positive]]"


In [ ]:
df_train['label'] = df_train['annotation'].str[0]
df_train['label'] = df_train['label'].str[-1]
df_train

,id,sentence_form,annotation,label
0,00001,둘쨋날은 미친듯이 밟아봤더니 기어가 헛돌면서 틱틱 소리가 나서 경악.,"[[본품#품질, [기어, 16, 18], negative]]",negative
1,00002,"이거 뭐 삐꾸를 준 거 아냐 불안하고, 거금 투자한 게 왜 이래.. 싶어서 정이 확...","[[본품#품질, [기어 텐션, 67, 72], negative]]",negative
2,00003,간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다.,"[[제품 전체#일반, [None, 0, 0], positive]]",positive
3,00004,샥이 없는 모델이라 일반 도로에서 타면 노면의 진동 때문에 손목이 덜덜덜 떨리고 이...,"[[제품 전체#일반, [샥이 없는 모델, 0, 8], neutral]]",neutral
4,00005,안장도 딱딱해서 엉덩이가 아팠는데 무시하고 타고 있다.,"[[본품#일반, [안장, 0, 2], negative]]",negative
...,...,...,...,...
2996,02997,(슬픔),"[[제품 전체#가격, [None, 0, 0], negative]]",negative
2997,02998,보드랍고 괜찮다!,"[[제품 전체#품질, [None, 0, 0], positive]]",positive
2998,02999,#일본 유니클로 질이 우리나라보다 좋은 것 같으면 기분 탓인가.......,"[[브랜드#일반, [유니클로, 4, 8], neutral]]",neutral
2999,03000,마지막으로 귀여워서 집어온 모자.,"[[제품 전체#디자인, [모자, 15, 17], positive]]",positive


In [ ]:
print(df_train.isnull().sum())

id               0
sentence_form    0
annotation       0
label            0
dtype: int64


In [ ]:
# 한글과 공백을 제외하고 모두 제거
df_train['sentence_form'] = df_train['sentence_form'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z ]","")
df_train[:5]

C:\Users\pnu\AppData\Local\Temp\ipykernel_1612\820094974.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_train['sentence_form'] = df_train['sentence_form'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z ]","")


,id,sentence_form,annotation,label
0,00001,둘쨋날은 미친듯이 밟아봤더니 기어가 헛돌면서 틱틱 소리가 나서 경악,"[[본품#품질, [기어, 16, 18], negative]]",negative
1,00002,이거 뭐 삐꾸를 준 거 아냐 불안하고 거금 투자한 게 왜 이래 싶어서 정이 확 떨어...,"[[본품#품질, [기어 텐션, 67, 72], negative]]",negative
2,00003,간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다,"[[제품 전체#일반, [None, 0, 0], positive]]",positive
3,00004,샥이 없는 모델이라 일반 도로에서 타면 노면의 진동 때문에 손목이 덜덜덜 떨리고 이...,"[[제품 전체#일반, [샥이 없는 모델, 0, 8], neutral]]",neutral
4,00005,안장도 딱딱해서 엉덩이가 아팠는데 무시하고 타고 있다,"[[본품#일반, [안장, 0, 2], negative]]",negative


In [ ]:
print(df_train.isnull().sum())

id               0
sentence_form    0
annotation       0
label            0
dtype: int64


In [ ]:
df_train['sentence_form'] = df_train['sentence_form'].str.replace('^ +', "") # 공백은 empty 값으로 변경

C:\Users\pnu\AppData\Local\Temp\ipykernel_1612\156374660.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_train['sentence_form'] = df_train['sentence_form'].str.replace('^ +', "") # 공백은 empty 값으로 변경


In [ ]:
df_test['sentence_form'] = df_test['sentence_form'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z ]","") # 정규 표현식 수행
df_test['sentence_form'] = df_test['sentence_form'].str.replace('^ +', "") # 공백은 empty 값으로 변경
df_test['sentence_form'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
df_test = df_test.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(df_test))

전처리 후 테스트용 샘플의 개수 : 2127


C:\Users\pnu\AppData\Local\Temp\ipykernel_1612\4260889883.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test['sentence_form'] = df_test['sentence_form'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z ]","") # 정규 표현식 수행
C:\Users\pnu\AppData\Local\Temp\ipykernel_1612\4260889883.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test['sentence_form'] = df_test['sentence_form'].str.replace('^ +', "") # 공백은 empty 값으로 변경


In [ ]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
okt = Okt()
okt.morphs('와 이런 것도 영화라고 차라리 뮤직비디오를 만드는 게 나을 뻔', stem = True)

['오다', '이렇다', '것', '도', '영화', '라고', '차라리', '뮤직비디오', '를', '만들다', '게', '나다', '뻔']

In [ ]:
X_train = []
for sentence in tqdm(df_train['sentence_form']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)

100%|█████████████████████████████████████████████████████████████████████████████| 3001/3001 [00:04<00:00, 686.82it/s]


In [ ]:
print(X_train[:3])

[['둘쨋', '날', '미치다', '밟다', '보다', '기어', '헛돌면', '서', '틱틱', '소리', '나서다', '경악'], ['거', '뭐', '삐', '꾸다', '준', '거', '알다', '불안하다', '거금', '투자', '게', '왜', '이렇다', '싶다', '정이', '확', '떨어지다', '산', '곳', '가져가다', '확인', '기어', '텐션', '문제', '라고', '고장', '아니다'], ['간사하다', '그', '이후', '에는', '라이딩', '아주', '즐겁다', '만족', '스럽게', '타다']]


In [ ]:
X_test = []
for sentence in tqdm(df_test['sentence_form']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_test.append(stopwords_removed_sentence)

100%|█████████████████████████████████████████████████████████████████████████████| 2127/2127 [00:03<00:00, 566.12it/s]


In [ ]:
# 따라한거
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 5418
등장 빈도가 2번 이하인 희귀 단어의 수: 3440
단어 집합에서 희귀 단어의 비율: 63.49206349206349
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 13.351287766220901


In [ ]:
# 전체 단어 개수 중 빈도수 2이하인 단어는 제거.
# 0번 패딩 토큰을 고려하여 + 1
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 1979


In [ ]:
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
print(X_train[:3])

[[173, 623, 12, 704, 215, 705, 796], [52, 387, 1492, 52, 344, 1030, 107, 490, 345, 89, 706, 624, 625, 330, 1493, 797, 704, 1031, 898, 1494, 113], [178, 1220, 298, 299, 565, 241, 421, 798]]


In [ ]:
y_train = np.array(df_train['label'])
y_test = np.array(df_test['label'])

In [ ]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

In [ ]:
# 빈 샘플들을 제거
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))
print(len(y_train))

2995
2995


C:\Users\pnu\anaconda3\lib\site-packages\numpy\lib\function_base.py:4454: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)


In [ ]:
def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

In [ ]:
max_len = 30
below_threshold_len(max_len, X_train)

전체 샘플 중 길이가 30 이하인 샘플의 비율: 99.26544240400668


In [ ]:
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=5, callbacks=[es, mc], batch_size=64, validation_split=0.2)

Epoch 1/5


UnimplementedError: Graph execution error:

Detected at node 'binary_crossentropy/Cast' defined at (most recent call last):
    File "C:\Users\pnu\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\pnu\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\pnu\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\pnu\AppData\Roaming\Python\Python39\site-packages\traitlets\config\application.py", line 978, in launch_instance
      app.start()
    File "C:\Users\pnu\AppData\Roaming\Python\Python39\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\pnu\AppData\Roaming\Python\Python39\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\pnu\anaconda3\lib\asyncio\base_even

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

out_nodes = len(y_test.value_counts())
loss_function = 'categorical_crossentropy'
activation_function = 'softmax'
input_size = X.shape[1]
y = to_categorical(y)
model = Sequential()

model.add(Dense(60, input_dim=input_size))
model.add(Activation('relu'))

model.add(Dense(40))
model.add(Activation('relu'))

model.add(Dense(20))
model.add(Activation('relu'))

model.add(Dense(out_nodes))
model.add(Activation(activation_function))

# 모델 컴파일
model.compile(loss=loss_function, optimizer='adam', metrics=['accuracy'])
model.summary()

AttributeError: 'numpy.ndarray' object has no attribute 'value_counts'

In [ ]:
# 학습 모델을 저장할 경로
path = './model/22'

# 만약 폴더가 있다면 삭제
if os.path.isdir(path):
    shutil.rmtree(path)

# 폴더 생성
os.makedirs(os.path.join(path))

In [ ]:
#
path1 = path + '/{epoch}-{val_loss}.h5'
path2 = path + '/best_model.h5'

# 저장콜백
# save_best_olny: True를 설정해주면 모니터링하는 값이 개선된 경우에만 저장
# 기본은 False 이고 False시 매번 저장
# monitor: 저장하기 위해서 모니터링할 값
call1 = ModelCheckpoint(filepath = path1, monitor = 'val_loss', save_best_olny=True)
call2 = ModelCheckpoint(filepath = path2, monitor = 'val_loss', save_best_olny=True)

In [ ]:
# 조기 중단
# monitor : 감시할 검증 값
# patience : 현재 평가 값이 전 평가 값보다 좋다고 판달될 경우 몇번 더 볼껀지 설정
call3 = EarlyStopping(monitor = 'val_loss', patience=50)

In [ ]:
# 학습
history = model.fit(X, y, epochs=15, batch_size = 64, validation_split=0.2, callbacks = [call1, call2, call3])

In [ ]:
from transformers import ElectraTokenizer
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
tokenizer.tokenize("[CLS] 한국어 ELECTRA를 공유합니다. [SEP]")
['[CLS]', '한국어', 'EL', '##EC', '##TRA', '##를', '공유', '##합니다', '.', '[SEP]']
tokenizer.convert_tokens_to_ids(['[CLS]', '한국어', 'EL', '##EC', '##TRA', '##를', '공유', '##합니다', '.', '[SEP]'])
[2, 11229, 29173, 13352, 25541, 4110, 7824, 17788, 18, 3]

In [ ]:
df_dev.shape, df_test.shape, df_train.shape

In [ ]:
# 버트
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

In [ ]:
# 토크나이저 획득
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

In [ ]:
from transformers import ElectraModel, ElectraTokenizer

model1 = ElectraModel.from_pretrained("monologg/koelectra-base-discriminator")  # KoELECTRA-Base
model2 = ElectraModel.from_pretrained("monologg/koelectra-small-discriminator")  # KoELECTRA-Small
model3 = ElectraModel.from_pretrained("monologg/koelectra-base-v2-discriminator")  # KoELECTRA-Base-v2
model4 = ElectraModel.from_pretrained("monologg/koelectra-small-v2-discriminator")  # KoELECTRA-Small-v2
model5 = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")  # KoELECTRA-Base-v3
model6 = ElectraModel.from_pretrained("monologg/koelectra-small-v3-discriminator")  # KoELECTRA-Small-v3
# model = TFElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator", from_pt=True)

In [ ]:
# 일렉트라 토크나이저 획득
from transformers import ElectraTokenizer
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

In [ ]:
# TODO 추가 0 => numpy 추가
import numpy as np

# 데이터 프레임 train_nli에서 컬럼 sentence1, sentence2 데이터를 순서대로 각각 추출한다 -> for ~
def data_convert( sen1s, max_length, tokeni ):
  '''
    sen1s : 문장으로 구성된 말뭉치 1번세트
    max_length : 토큰화 과정시 문장 최대길이 지정
    tokeni : 토큰화 도구(토크나이저)
  '''
  # 각각 데이터를 담을 그릇
  input_ids       = list()
  token_type_ids  = list()
  attention_masks = list()

  for s1 in tqdm(zip(sen1s), total=len(sen1s) ):
    # 인코딩 처리
    # TODO 수정 1 => pad_to_max_length
    results = tokeni.encode_plus( s1, max_length=max_length, pad_to_max_length=True )
    # 각각 데이터들을 그릇에 추가한다
    input_ids.append(       results['input_ids'] )
    token_type_ids.append(  results['token_type_ids'] )
    attention_masks.append( results['attention_mask'] )
    pass

  # TODO 추가 1 배열 형태로 변환하여 문장 1개당 128개 백터로 표현되게 처리
  input_ids       = np.array(input_ids, dtype=int)
  token_type_ids  = np.array(token_type_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)

  return input_ids, token_type_ids, attention_masks

In [ ]:
# 샘플링으로 1개씩만 확인
sent1 = df_train.sentence_form.iloc[0]

In [ ]:
# 문장의 최대 길이 지정(토큰화시)
max_length = 128

sample_encode = tokenizer.encode_plus(sent1,
                                      max_length=max_length, pad_to_max_length=True  )

In [ ]:
sample_encode
# input_ids     : 토큰의 사전상의 인덱스로 표기
#                 0, 1, 2, 3, ... => 스페셜 토큰 [CLS]:문장이시작한다, [SEP]:문장이끝났다, ...
#                 [UNK]:모르는 단어다. 사전에 등록이 않되어 있다
#                 [MASK]:마스크 토큰(숨김용도), [PAD]:문장 크기 보정용
# token_type_ids: 첫번째 문장은 0, 두번째 문장은 1 : 문장 구분용
# attention_mask: 문장이면 1, 패딩이면 0

In [ ]:
# 장시간 작업시 진행율을 보여주는 기능 -> tqdm
from tqdm import tqdm

for s1 in tqdm(zip(df_train.sentence_form), total=len(df_train.sentence_form) ):
  pass
  #print( s1 )
  #break

In [ ]:
# TODO 추가 0 => numpy 추가
import numpy as np

# 데이터 프레임 train_nli에서 컬럼 sentence1, sentence2 데이터를 순서대로 각각 추출한다 -> for ~
def data_convert( sen1s, max_length, tokeni ):
  '''
    sen1s : 문장으로 구성된 말뭉치 1번세트
    max_length : 토큰화 과정시 문장 최대길이 지정
    tokeni : 토큰화 도구(토크나이저)
  '''
  # 각각 데이터를 담을 그릇
  input_ids       = list()
  token_type_ids  = list()
  attention_masks = list()

  for s1 in tqdm(zip(sen1s), total=len(sen1s) ):
    # 인코딩 처리
    # TODO 수정 1 => pad_to_max_length
    results = tokeni.encode_plus( s1, max_length=max_length, pad_to_max_length=True )
    # 각각 데이터들을 그릇에 추가한다
    input_ids.append(       results['input_ids'] )
    token_type_ids.append(  results['token_type_ids'] )
    attention_masks.append( results['attention_mask'] )
    pass

  # TODO 추가 1 배열 형태로 변환하여 문장 1개당 128개 백터로 표현되게 처리
  input_ids       = np.array(input_ids, dtype=int)
  token_type_ids  = np.array(token_type_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)

  return input_ids, token_type_ids, attention_masks

In [ ]:
X_train = data_convert( df_train.sentence_form, max_length, tokenizer )

In [ ]:
# 결과 형태 확인
len(X_train), type( X_train[0] ), type( X_train[1] ), type( X_train[2] )

In [ ]:
# 검증용, 테스트용 컨버전 모두 진행
X_val  = data_convert( df_dev.sentence_form, max_length, tokenizer )
X_test = data_convert( df_test.sentence_form, max_length, tokenizer )

In [ ]:
print( 'input_ids', X_test[0][0] )

In [ ]:
print( 'token_type_ids', X_test[1][0] )

In [ ]:
print( 'attention_mask', X_test[2][0] )

In [ ]:
# 문장 복원
tokenizer.decode( X_test[0][0] )

In [ ]:
train_labels = df_train['annotation'].tolist()
val_labels   = df_dev['annotation'].tolist()
test_labels  = df_test['annotation'].tolist()

In [ ]:
# LabelEncoder : 정답을 정수로 변환 처리
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_enc = LabelEncoder()

In [ ]:
# 정답 대비 정수값 세트 획득
label_enc.fit( train_labels )

In [ ]:
# 변환처리
y_train = label_enc.transform( train_labels )
y_val   = label_enc.transform( val_labels )
y_test  = label_enc.transform( test_labels )